In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import pmdarima as pm
from sklearn.neural_network import MLPRegressor

In [7]:
df = (
    pd.read_csv("data/analysis_data/merged_data_US.csv", parse_dates=["month"])
      .rename(columns=lambda s: s.strip().lower().replace(" ", "_"))
      .set_index("month")
)
state = "California"  # ← change or loop this
sub = df[df.state == state]

ts = sub.unemployment_rate.asfreq("MS")
n_train = int(len(ts) * 0.85)
train_y = ts.iloc[:n_train]
test_y  = ts.iloc[n_train:]
train_exog = sub[['median_income','population','initial_claims','lfp_rate']].iloc[:n_train]
test_exog  = sub[['median_income','population','initial_claims','lfp_rate']].iloc[n_train:]

In [8]:
sarimax = pm.auto_arima(
    train_y,
    exogenous=train_exog,
    seasonal=True, m=12,
    start_p=0, max_p=5, start_q=0, max_q=5,
    start_P=0, max_P=2, start_Q=0, max_Q=2,
    d=None, D=1,
    trace=True,
    error_action='ignore',
    suppress_warnings=True
)

# 4. Extract in‑sample residuals
resid = pd.Series(sarimax.resid(), index=train_y.index)

# helper to build lagged features
def make_lagged_matrix(series, lags):
    X, y = [], []
    vals = series.values
    for i in range(lags, len(vals)):
        X.append(vals[i-lags:i])
        y.append(vals[i])
    return np.array(X), np.array(y)

LAGS = 12
X_res, y_res = make_lagged_matrix(resid, LAGS)

c:\Users\nelli\anaconda3\envs\new_env\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\nelli\anaconda3\envs\new_env\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\nelli\anaconda3\envs\new_env\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\nelli\anaconda3\envs\new_env\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\nelli\anaconda3\envs\new_env\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finit

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,0)[12] intercept   : AIC=1240.748, Time=0.42 sec


c:\Users\nelli\anaconda3\envs\new_env\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(1,0,0)(1,1,0)[12] intercept   : AIC=inf, Time=0.60 sec


c:\Users\nelli\anaconda3\envs\new_env\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

 ARIMA(3,0,1)(2,1,0)[12]             : AIC=-883.386, Time=2.87 sec


c:\Users\nelli\anaconda3\envs\new_env\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(2,0,1)(2,1,0)[12]             : AIC=-848.761, Time=2.83 sec


c:\Users\nelli\anaconda3\envs\new_env\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(4,0,1)(2,1,0)[12]             : AIC=-883.130, Time=4.58 sec

Best model:  ARIMA(3,0,0)(2,1,0)[12]          
Total fit time: 114.350 seconds


In [9]:
mlp = MLPRegressor(
    hidden_layer_sizes=(16,),
    activation='relu',
    solver='adam',
    max_iter=500,
    early_stopping=True,
    validation_fraction=0.1,
    n_iter_no_change=10,
    random_state=42
)
mlp.fit(X_res, y_res)

h = len(test_y)

arima_forecast = sarimax.predict(n_periods=h, exogenous=test_exog)

last_resids = resid.values[-LAGS:].tolist()
mlp_preds = []
for _ in range(h):
    x_in = np.array(last_resids[-LAGS:]).reshape(1, -1)
    pred = mlp.predict(x_in)[0]
    mlp_preds.append(pred)
    last_resids.append(pred)

c:\Users\nelli\anaconda3\envs\new_env\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [10]:
combined = arima_forecast + np.array(mlp_preds)

print("ARIMA alone   → RMSE: {:.3f}, R²: {:.3f}".format(
    np.sqrt(mean_squared_error(test_y, arima_forecast)),
    r2_score(test_y, arima_forecast)))

print("Hybrid ARIMA+MLP → RMSE: {:.3f}, R²: {:.3f}".format(
    np.sqrt(mean_squared_error(test_y, combined)),
    r2_score(test_y, combined)))

ARIMA alone   → RMSE: 3.427, R²: -0.536
Hybrid ARIMA+MLP → RMSE: 3.429, R²: -0.538
